**Как и оговорено условием задачи, часть кода я удалил, но оставил часть с вызовом pip и импортом библиотек(я строго следую условиям задачи)**

In [1]:
pip --version
pip install pyspark
pip install faker

pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [2]:
# импортирую необходимые библиотеки в одном блоке
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, to_date

In [5]:

# Создание SparkSession
spark = SparkSession.builder.appName("Read CSV Example").getOrCreate()

# Чтение CSV-файла
df = spark.read.csv("web_server_logs.csv", header=True, inferSchema=True)

# Печать схемы DataFrame
df.printSchema()

# Показ первых 5 строк
df.show(5)

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)

+---------------+--------------------+------+--------------------+-------------+-------------+
|             ip|           timestamp|method|                 url|response_code|response_size|
+---------------+--------------------+------+--------------------+-------------+-------------+
|  14.69.254.151|2025-01-05 21:22:...|   GET|wp-content/tag/ca...|          404|         5939|
|132.238.192.133|2025-01-02 12:31:...|DELETE|   tags/list/explore|          404|         8304|
| 36.155.231.139|2025-01-02 07:37:...|DELETE|      main/blog/tags|          200|         8370|
| 104.221.81.249|2025-01-02 06:06:...|  POST|                main|          301|         5746|
|  108.73.51.230|2025-01-05 21:55:...|  POST|        tag/category|          404|      

In [16]:
# Решаю пункт 1(подсчет по ip), соритрую по убыванию
df.groupBy("ip").agg(F.count("ip").alias("request_count")).sort("request_count", ascending=False).show(10)


+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
| 208.37.170.254|            2|
|    9.89.69.181|            2|
|125.232.117.134|            1|
|  145.100.95.96|            1|
| 55.184.149.115|            1|
|   22.96.187.25|            1|
|203.102.191.157|            1|
|  28.80.144.151|            1|
|175.247.215.122|            1|
| 177.120.78.127|            1|
+---------------+-------------+
only showing top 10 rows



In [19]:
# Решаю пункт 2(подсчет по методу), соритрую по убыванию
df.groupBy("method").agg(F.count("method").alias("method_count")).sort("method_count",ascending=False).show()

+------+------------+
|method|method_count|
+------+------------+
|   PUT|       25256|
|  POST|       25004|
|   GET|       24960|
|DELETE|       24780|
+------+------------+



In [22]:
# Решаю п.3: Профильтруйте и посчитайте количество запросов с кодом ответа 404
mvar = df.select('response_code').where(df.response_code == 404).count()
print(f"Number of 404 response codes:{mvar}")


Number of 404 response codes:25231


In [28]:
# Решаю п.4:  Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.

df = df.withColumn('date', to_date(col('timestamp')))
df.groupBy("date").agg(F.sum("response_size").alias("total_response_size")).sort("date",ascending=True).show()


+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2025-01-01|           73942369|
|2025-01-02|           72358989|
|2025-01-03|           74089472|
|2025-01-04|           73588518|
|2025-01-05|           73417217|
|2025-01-06|           73521589|
|2025-01-07|           66055545|
+----------+-------------------+

